# Lab 002 · Random Forest (Rosario) — Folium / Thebe

Este cuaderno usa **geemap.foliumap** (no widgets) para que los mapas se vean en Thebe/Binder.

In [ ]:
# === Celda 0: asegurar paquetes y forzar backend folium ===
import importlib, sys, subprocess
def ensure(pkgs):
    miss = [p for p in pkgs if importlib.util.find_spec(p) is None]
    if miss:
        subprocess.check_call([sys.executable, "-m", "pip", "install", *miss])
ensure(["earthengine-api", "geemap", "folium"])

# Limpiar módulos previos de geemap (por si quedó el backend de widgets cargado)
for _m in list(sys.modules):
    if _m.startswith("geemap"):
        del sys.modules[_m]

import ee
import importlib as _imp
geemap = _imp.import_module("geemap.foliumap")  # <<-- backend folium SIEMPRE
from IPython.display import display
print("geemap backend:", geemap.__name__)


In [ ]:
import importlib
print("ee:", bool(importlib.util.find_spec("ee")))
print("geemap.foliumap:", bool(importlib.util.find_spec("geemap.foliumap")))
print("folium:", bool(importlib.util.find_spec("folium")))

In [ ]:
# === Celda 1: Autenticación / Inicialización de Earth Engine ===

# === Celda 1: Imports + inicialización de Earth Engine (autónoma) ===
import ee
import geemap.foliumap as geemap   # SIEMPRE este backend (no widgets)
from IPython.display import display

print("geemap backend:", geemap.__name__)  # Debe decir geemap.foliumap

try:
    ee.Initialize(project="ee-cdgidera")
    print("EE inicializado.")
except Exception as e:
    print("Autenticación requerida. Seguí el enlace, pegá el código y re-ejecutá esta celda.")
    ee.Authenticate()
    ee.Initialize(project="ee-cdgidera")


In [ ]:
# === Celda 2: Cargar assets (ROI y GCPS) ===
roi_fc = ee.FeatureCollection('users/cdg-idera/roi_polygon_2024')
roi = roi_fc.geometry()
gcps_fc = ee.FeatureCollection('users/cdg-idera/gcps_landcover_2024')
print('ROI features:', roi_fc.size().getInfo())
print('GCPS features:', gcps_fc.size().getInfo())


In [ ]:
# === Celda 3: Sentinel-2: filtro -> composite (mediana) ===
s2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
filtered = (s2
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 30))
    .filterDate('2024-01-01', '2025-01-01')
    .filterBounds(roi)
    .select('B.*')
)
composite = filtered.median().clip(roi)
rgbVisParams = {'min': 0, 'max': 3000, 'bands': ['B4','B3','B2']}

Map = geemap.Map()              # backend folium
Map.centerObject(roi, 10)
Map.addLayer(composite, rgbVisParams, 'Composite S2 2024 (ROI)')
display(Map)


In [ ]:
# === Celda 4: Partición 60/40 train/valid ===
gcp = gcps_fc.randomColumn()
trainingGCP = gcp.filter(ee.Filter.lt('random', 0.6))
validationGCP = gcp.filter(ee.Filter.gte('random', 0.6))
print('Train pts:', trainingGCP.size().getInfo(), '| Valid pts:', validationGCP.size().getInfo())


In [ ]:
# === Celda 5: Sampleo de pixeles para entrenamiento (FIX PYTHON) ===
training = composite.sampleRegions(
    collection=trainingGCP,
    properties=['landcover'],
    scale=10,
    tileScale=16
)
print('Training samples:', training.size().getInfo())

In [ ]:
# === Celda 6: Entrenamiento del clasificador (Random Forest) ===
classifier = ee.Classifier.smileRandomForest(100).train(
    features=training,
    classProperty='landcover',
    inputProperties=composite.bandNames()
)
print('Clasificador entrenado.')


In [ ]:
# === Celda 7: Clasificar e imagen clasificada ===
classified = composite.classify(classifier)
classVis = {'min': 0, 'max': 4, 'palette': ['blue', 'gray', 'green', 'violet', 'orange']}

Map2 = geemap.Map()
Map2.centerObject(roi, 10)
Map2.addLayer(classified.clip(roi), classVis, 'Imagen Clasificada')
display(Map2)


In [ ]:
# === Celda 8: Evaluación de exactitud ===
# === Test set / evaluación (FIX PYTHON) ===
test = classified.sampleRegions(
    collection=validationGCP,
    properties=['landcover'],
    scale=10
)
cm = test.errorMatrix('landcover', 'classification')
print("Confusion Matrix:\n", cm.getInfo())
print("Overall Accuracy:", cm.accuracy().getInfo())
print("Producers Accuracy:", cm.producersAccuracy().getInfo())
print("Consumers Accuracy:", cm.consumersAccuracy().getInfo())
print("Kappa:", cm.kappa().getInfo())
print("F1 (class 1):", cm.fscore(1).getInfo())

